## Parameters description for the unmixing experiments
Train and evaluate the convolutional neural network (CNN)
as well as the deep convolutional autoencoder (DCAE)
for the unmixing problem. <br>
Exemplary hyperparameters to set:<br>
- --data-file-path - path to the hyperspectral image (HSI).
- --ground-truth-path - path to the ground truth map containing
the fractions of abundances for entire HSI.
- --train-size - magnitude of the learning set that is utilized
to fine-tune the weights of the model.
- --sub-test-size - size of the test set to evaluate
the generalization of the model. It is sampled from the remaining
HSI excluding the training subset. If not specified, all non-training samples
constitute the test set.<br>Can be employed in the case of experiments
when changing the magnitudes of training sets but keeping the size of test
set constant.
- --val-size - fraction or size of the validation subset, it is designed to
monitor the overfitting.
- --channels-idx - index of the spectral dimension in input HSI.
- batch-size - number of samples per update step in the training phase.
- --shuffle - indicates whether to shuffle the dataset in experiment.
- --patience - stopping condition for a specific number of epochs without
improvement.
- --model-name - name of the utilized model, exemplary values:<br>
unmixing_pixel_based_cnn, unmixing_cube_based_cnn, unmixing_pixel_based_dcae,
unmixing_cube_based_dcae for the pixel-based, cube-based CNN and DCAE
respectively.
- --sample-size - number of spectral bands in a given HSI.
- --neighborhood-size - size of the spatial extent which is employed for each
sample in the form of local neighboring pixels. Most cases allows to leverage
the quality of the segmentation as well as the unmixing.
- --n-classes - number of endmembers in the HSI for which the abundances
will be estimated by the model.
- --lr - learning rate, regulates the step size during weights updates.
- --epochs - second stopping condition, i.e., the maximum number of epochs.
- --verbose - verbosity mode.
- --save-data - indicates whether to save the training and test data.

## Cube-based DCAE
We specify the necessary parameters for the experiment.

In [1]:
# Execute cube-based DCAE:
from os.path import join

base_path = r'../datasets/urban'
data_file_path = join(base_path, 'urban.npy')
ground_truth_path = join(base_path, 'urban_gt.npy')
endmembers_path = join(base_path, 'urban_m.npy')
train_size = 15500
sub_test_size = 47249
val_size = 0.1
channels_idx = -1
batch_size = 256
shuffle = True
patience = 3
model_name = 'unmixing_cube_based_dcae'
sample_size = 162
neighborhood_size = 5
n_classes = 6
dest_path = join('../examples', 'unmixing_results')
lr = 0.0005
epochs = 10
verbose = 2
save_data = False
use_mlflow = False
seed = 1

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf

from ml_intuition import enums
from ml_intuition.data.utils import parse_train_size, subsample_test_set
from scripts import prepare_data
from scripts.unmixing import train_unmixing, evaluate_unmixing
from scripts.unmixing.unmixing_experiments_runner import NEIGHBORHOOD_SIZES, \
    LEARNING_RATES

dcae_dest_path = join(dest_path, 'cube-based-dcae')
os.makedirs(dcae_dest_path, exist_ok=True)

# Apply default literature hyperparameters:
if neighborhood_size is None and model_name in NEIGHBORHOOD_SIZES:
    neighborhood_size = NEIGHBORHOOD_SIZES[model_name]
if lr is None and model_name in LEARNING_RATES:
    lr = LEARNING_RATES[model_name]

## Prepare data
Prepare the data for unmixing utilizing the *prepare_data.main* method.<br>
It accepts various parameters such as path to the data file or ground-truth
for a specific HSI. Furthermore, magnitude of the learning set can be specified.
The method accepts also the neighborhood size parameter which specifies the
spatial extent of ech sample.

In [3]:
# Prepare data for unmixing:
data = prepare_data.main(data_file_path=data_file_path,
                         ground_truth_path=ground_truth_path,
                         train_size=parse_train_size(train_size),
                         val_size=val_size,
                         stratified=False,
                         background_label=-1,
                         channels_idx=channels_idx,
                         neighborhood_size=neighborhood_size,
                         save_data=save_data,
                         seed=seed,
                         use_unmixing=True)
# Subsample the test set to constitute a constant size:
if sub_test_size is not None:
    subsample_test_set(data[enums.Dataset.TEST], sub_test_size)

## Train and evaluate the model
Few parameters previously initialized are employed in this step, e.g.,
the name of the model, size of the spectral extent,
learning rate and batch size.
The results including the metrics are stored in *dcae_dest_path* directory.

In [4]:
# Train the model:
train_unmixing.train(model_name=model_name,
                     dest_path=dcae_dest_path,
                     data=data,
                     sample_size=sample_size,
                     neighborhood_size=neighborhood_size,
                     n_classes=n_classes,
                     lr=lr,
                     batch_size=batch_size,
                     epochs=epochs,
                     verbose=verbose,
                     shuffle=shuffle,
                     patience=patience,
                     endmembers_path=endmembers_path,
                     seed=seed)
# Evaluate the model:
evaluate_unmixing.evaluate(
    model_path=os.path.join(dcae_dest_path, model_name),
    data=data,
    dest_path=dcae_dest_path,
    neighborhood_size=neighborhood_size,
    batch_size=batch_size,
    endmembers_path=endmembers_path)
tf.keras.backend.clear_session()

c:\users\łukasz\desktop\machine-learning\venv\lib\site-packages\tensorflow\python\framework\tensor_util.py:125: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  tensor_proto.float_val.extend([np.asscalar(x) for x in proto_values])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 3, 3, 160, 16)     448       
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 1, 1, 158, 32)     13856     
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 1, 1, 156, 64)     6208      
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 1, 1, 154, 128)    24704     
_________________________________________________________________
flatten (Flatten)            (None, 19712)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               5046528   
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
__________

## Pixel-based CNN
We perform the same steps for the pixel-based CNN model, however few parameters
must be altered. Since we utilize only the spectral dimension, the
*neighborhood_size* is set to *None*, the learning rate is also adjusted.

In [5]:
# Execute the pixel-based CNN:
endmembers_path = None
model_name = 'unmixing_pixel_based_cnn'
lr = 0.01
neighborhood_size = None
cnn_dest_path = join(dest_path, 'pixel-based-cnn')
os.makedirs(cnn_dest_path, exist_ok=True)

# Apply default literature hyperparameters:
if neighborhood_size is None and model_name in NEIGHBORHOOD_SIZES:
    neighborhood_size = NEIGHBORHOOD_SIZES[model_name]
if lr is None and model_name in LEARNING_RATES:
    lr = LEARNING_RATES[model_name]
# Prepare data for unmixing:
data = prepare_data.main(data_file_path=data_file_path,
                         ground_truth_path=ground_truth_path,
                         train_size=parse_train_size(train_size),
                         val_size=val_size,
                         stratified=False,
                         background_label=-1,
                         channels_idx=channels_idx,
                         neighborhood_size=neighborhood_size,
                         save_data=save_data,
                         seed=seed,
                         use_unmixing=True)
# Subsample the test set to constitute a constant size:
if sub_test_size is not None:
    subsample_test_set(data[enums.Dataset.TEST], sub_test_size)
# Train the model:
train_unmixing.train(model_name=model_name,
                     dest_path=cnn_dest_path,
                     data=data,
                     sample_size=sample_size,
                     neighborhood_size=neighborhood_size,
                     n_classes=n_classes,
                     lr=lr,
                     batch_size=batch_size,
                     epochs=epochs,
                     verbose=verbose,
                     shuffle=shuffle,
                     patience=patience,
                     endmembers_path=endmembers_path,
                     seed=seed)
# Evaluate the model:
evaluate_unmixing.evaluate(
    model_path=os.path.join(cnn_dest_path, model_name),
    data=data,
    dest_path=cnn_dest_path,
    neighborhood_size=neighborhood_size,
    batch_size=batch_size,
    endmembers_path=endmembers_path)
tf.keras.backend.clear_session()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 1, 1, 158, 3)      18        
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 1, 1, 79, 3)       0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 1, 1, 76, 6)       78        
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 1, 1, 38, 6)       0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 1, 1, 34, 12)      372       
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 1, 1, 17, 12)      0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 1, 1, 14, 24)      1176      
__________